<a href="https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/calibration/self_calibration_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In Development

(https://casaguides.nrao.edu/index.php/First_Look_at_Imaging) will be used. 

This walkthrough is designed to be run in a Jupyter notebook on Google Colaboratory. To open the notebook in colab, go [here](https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/calibration/self_calibration_solve.ipynb).

## Installation and Dataset Download


In [2]:
import os
#os.system("pip install --extra-index-url https://test.pypi.org/simple/ cngi-prototype==0.0.70")

#sis14_twhya_field_5_lsrk.vis.zarr https://drive.google.com/file/d/1QRHqrYN_9jhGx0s0URzTYUVu5JsjJCEl/view?usp=sharing
id = '1QRHqrYN_9jhGx0s0URzTYUVu5JsjJCEl'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')



print('complete') 

complete


## Dataset

https://casaguides.nrao.edu/index.php?title=First_Look_at_Self_Calibration_CASA_6

How to recreate data:

https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar

```python
mstransform('sis14_twhya_calibrated_flagged.ms',
             outputvis='sis14_twhya_field_5_lsrk.ms',
             regridms=True, outframe='LSRK', datacolumn='corrected', field='5')

tclean(vis='sis14_twhya_field_5_lsrk.ms/',
       imagename='first_image',
       spw='',
       specmode='mfs',
       deconvolver='hogbom',
       nterms=1,
       gridder='standard',
       imsize=[250,250],
       cell=['0.1arcsec'],
       weighting='natural',
       threshold='0mJy',
       niter=5000,
       savemodel='modelcolumn')

from cngi.conversion import convert_ms
vis_mxds = convert_ms('sis14_twhya_field_5_lsrk.ms/',chunks=(27,21,32,2))
```

In [10]:
#Reload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import xarray as xr
from cngi.dio import read_vis, read_image
from ngcasa.calibration import solve_calibration
from cngi.vis import chan_average, time_average
#from cngi.vis import apply_flag

xr.set_options(display_style="html")
# 270, 210, 384, 2
vis_mxds = read_vis("sis14_twhya_field_5_lsrk.vis.zarr",chunks={'time':27,'baseline':210,'chan':384,'pol':2})

x = time_average(vis_mxds, 'xds0', bin=28)
#(mxds, vis, bin=1, width=None, span='state', maxuvwdistance=None)[source]
print(x.xds0)

#print(vis_mxds.xds0)

cal_xds = None
sel_parms = {}
sel_parms['xds'] = 'xds0'
sel_parms['data_group_in'] = {'data':'DATA','model_data':'MODEL_DATA','weight':'WEIGHT'}

solve_parms= {}



#cal_solution = solve_calibration(vis_mxds, cal_xds, solve_parms, sel_parms)


#cal_solution.compute()


/Users/jsteeb/anaconda3/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/jsteeb/anaconda3/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in greater
  return func(*(_execute_task(a, cache) for a in args))


<xarray.Dataset>
Dimensions:         (baseline: 210, chan: 384, pol: 2, pol_id: 1, spw_id: 1, time: 11, uvw_index: 3)
Coordinates:
  * chan            (chan) float64 3.725e+11 3.725e+11 ... 3.728e+11 3.728e+11
    chan_width      (chan) float64 dask.array<chunksize=(384,), meta=np.ndarray>
    effective_bw    (chan) float64 dask.array<chunksize=(384,), meta=np.ndarray>
  * pol             (pol) int32 9 12
  * pol_id          (pol_id) int32 0
    resolution      (chan) float64 dask.array<chunksize=(384,), meta=np.ndarray>
  * spw_id          (spw_id) int32 0
  * time            (time) datetime64[ns] 2012-11-19T07:57:59.892000062 ... 2...
  * baseline        (baseline) int64 0 1 2 3 4 5 6 ... 204 205 206 207 208 209
Dimensions without coordinates: uvw_index
Data variables:
    ANTENNA1        (time, baseline) int32 dask.array<chunksize=(1, 210), meta=np.ndarray>
    ANTENNA2        (time, baseline) int32 dask.array<chunksize=(1, 210), meta=np.ndarray>
    ARRAY_ID        (time, baseline)